# Team Members and Contributions
![Contributions](https://github.com/SHIROKAMIQQ/cs138project/blob/main/contributions.png?raw=true)

# Pledge

![Pledge](https://github.com/SHIROKAMIQQ/cs138project/blob/main/pledge.png?raw=true)

# Summary

Due to its position in the Ring of Fire, the Philippines experiences a great number of earthquakes every year, causing severe structural damage and loss of lives. As such, finding a way to model earthquakes in the country can help in identifying where the most vulnerable parts are and preparing for the next incident.

For this project, we will be mapping reports of earthquake magnitude and intensity across various parts of the country during the Magnitude 7.6 Earthquake in Davao that occurred on December 2023.


# Methods

The project can be divided onto three parts,
- Data preprocessing
- Synthetic Data Generation
- Vulnerability Assessment
- Accuracy Testing through Spatio-temporal Analysis

## Data Gathering and Preprocessing

The project will use three datasets for the mapping of the earthquake: the Did You Feel It Survey from the United States Geological Survey (USGS), the Android alerts from the Earthquake Notification Service sourced from Zenodo, as well as the PHIVOLCS Earthquake Bulletins.

We first condense the USGS dataset to a period with the most earthquakes

### Condensing by Time

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

In [ ]:
df_usgs = pd.read_csv("https://raw.githubusercontent.com/SHIROKAMIQQ/cs138project/refs/heads/main/USGS_DATA.csv")
df_usgs.head()

In [ ]:
df_usgs['time'] = df_usgs['time'].str[:10]
df_usgs.head()

In [ ]:
df_counts = df_usgs['time'].value_counts()
df_counts

Plot the dataframe to observe trends

In [ ]:
import plotly.express as px

fig = px.scatter(df_counts, x = df_counts.index, y = df_counts, title='Number of Philippine Earthquakes')
fig.show()

From the seen trend, the data shall be condensed onto the earthquakes that had only occurred during the time period December 2, 2023 until December 12, 2023.

This time range was considered as it contained the most amount of earthquakes during a 10-day period condensed on that area, with 678 recorded latitude-longitude points.

This period was also considered from the continual magnitude 4-6 earthquakes that occurred during the said 10-day period after the initial 7.6 magnitude earthquake.

In [ ]:
df_2023_12 = df_usgs[((df_usgs['time'] > '2023-12-01') & (df_usgs['time'] < '2023-12-13')) & df_usgs['place'].str.contains('Philippines')]
df_2023_12.describe()

In [ ]:
df_2023_12.head()

In [ ]:
df_2023_12.to_csv('202312Dataset.csv', index=None)

### Condensing by Space
#### Spatial Clustering of Relevant Points

From this, we had then spatially clustered the dataset from USGS. This is done by taking the haversine distance of each point/row and applying the DBSCAN clustering algorithm (Boeing, 2014), where the maximum distance between neighbors is 30 kilometers. Within this, we also fitted the cluster into a square, which will help us make a square matrix for further processes. 

This is to be used for the area to be compared to the L by L matrix made by the Synthetic Data Later on

In [ ]:
INPUT_FILE = "202312Dataset.csv"
OUTPUT_FILE = "202312Spacial.csv"

# This acts as the threshold for considering how close cells must be to each other
#  to be considered part of the same neighborhood/cluster.
EPS_KM = 30
EARTH_RADIUS_KM = 6371.0
EPS_RAD = EPS_KM / EARTH_RADIUS_KM

# Load .csv file
# This assumes dyfi for now. 
csv_file = INPUT_FILE  # replace with your file path
df = pd.read_csv(csv_file)
coords = df[['latitude', 'longitude']].to_numpy()

# DBSCAN clustering using Haversine distance
# https://stackoverflow.com/questions/24762435/clustering-geo-location-coordinates-lat-long-pairs-using-kmeans-algorithm-with
# https://geoffboeing.com/2014/08/clustering-to-reduce-spatial-data-set-size/ 
coords_rad = np.radians(coords)  # lat/lon in radians
db = DBSCAN(eps=EPS_RAD, min_samples=3, metric='haversine')
# print("DB:", db)
labels = db.fit_predict(coords_rad)
# print("labels:", labels)
df['Cluster'] = labels

# Find largest cluster/neighborhood
unique_labels = [l for l in set(labels) if l != -1]  # exclude noise
if not unique_labels:
    raise ValueError("No clusters found. Try increasing eps or decreasing min_samples.")
cluster_sizes = {l: np.sum(labels == l) for l in unique_labels}
largest_cluster_label = max(cluster_sizes, key=cluster_sizes.get)
print(f"Largest cluster: {largest_cluster_label} with {cluster_sizes[largest_cluster_label]} points")

# Extract rows of largest cluster to dyfi_largest_cluster.csv
largest_cluster_rows = df[df['Cluster'] == largest_cluster_label]
largest_cluster_rows.to_csv(OUTPUT_FILE, index=False)
print(f"Saved largest cluster to {OUTPUT_FILE}")

# Compute square in terms of lat/lon
lat_min = largest_cluster_rows['latitude'].min()
lat_max = largest_cluster_rows['latitude'].max()
lon_min = largest_cluster_rows['longitude'].min()
lon_max = largest_cluster_rows['longitude'].max()
lat_range = lat_max - lat_min
lon_range = lon_max - lon_min
max_range = max(lat_range, lon_range)

# Build square bounding box (equal sides)
lat_mid = (lat_max + lat_min)/2
lon_mid = (lon_max + lon_min)/2
half_size = max_range/2

square_lat_min = lat_mid - half_size
square_lat_max = lat_mid + half_size
square_lon_min = lon_mid - half_size
square_lon_max = lon_mid + half_size

print("Square bounding box (lat/lon degrees):")
print(f"latitude:  {square_lat_min:.4f} to {square_lat_max:.4f}")
print(f"longitude: {square_lon_min:.4f} to {square_lon_max:.4f}")

# Plot
plt.figure(figsize=(8,6))
colors = plt.cm.get_cmap('tab20', len(unique_labels))

for k in unique_labels:
    mask = (labels == k)
    xy = coords[mask]
    plt.scatter(xy[:,1], xy[:,0], label=f'Cluster {k}')

# noise points
mask_noise = (labels == -1)
plt.scatter(coords[mask_noise,1], coords[mask_noise,0], c='k', marker='x', label='Noise')

# overlay square of largest cluster
square_lon = [square_lon_min, square_lon_max, square_lon_max, square_lon_min, square_lon_min]
square_lat = [square_lat_min, square_lat_min, square_lat_max, square_lat_max, square_lat_min]
plt.plot(square_lon, square_lat, 'r-', linewidth=2, label='Bounding Square (Largest Cluster)')

plt.xlabel('longitude')
plt.ylabel('latitude')
plt.title('DBSCAN Clustering of DYFI Responses with Square Bounding Box')
plt.legend()
plt.show()

## Synthetic Data Generation

A modified Olami-Feder-Christensen (OFC) model will be used to simulate synthetic earthquakes and produce data from different simulated intensities, adapted from (Greco et al., 2019) and (Ferreira et al., 2022). Which is a cellular automaton model that starts with an $L \times L$ matrix wherein for each cell (i, j) inside the matrix, there corresponds some seismologic force $F$. This aims to mimic the uniform motion of tectonic plates, the $F_{t h}$ is a threshold value that indicates the limit of the friction force.

This said model is said to reproduce the statistical features of earthquakes which shall be suited for vulnerability assessment for urban areas.

In [ ]:
from collections import deque
from random import random, randint
from math import log

class OFCModel:
    def __init__(self, L: int, ALPHA: float, F_TH: float, f_0: list[list[float]], p: int) -> None:
        self.L = L
        self.ALPHA = ALPHA
        self.F_TH = F_TH
        self.f_0 = self._deepcopy_2d(f_0)
        self.p = p
        self.adj: dict[tuple[int, int], list[tuple[int, int]]] = dict()

        for i in range(L):
            for j in range(L):
                self.adj[(i,j)] = []
                for (di,dj) in ((-1,0),(1,0),(0,-1),(0,1)):
                    ni, nj = i+di, j+dj
                    if 0 <= ni < L and 0 <= nj < L:
                        self.adj[(i,j)].append((ni,nj))
    
    def _deepcopy_2d(self, f: list[list[float]]) -> list[list[float]]:
        return [[cell for cell in row] for row in f]
    
    def _deepcopy_list_tuple(self, lt: list[tuple[int, int]]):
        return [(i, j) for i, j in lt]

    def rewire(self, adj: dict[tuple[int, int], list[tuple[int, int]]]):
        for i, j in adj:
            pi = randint(0, 99)
            if pi < self.p:
                index_to_change = randint(0, len(adj[(i, j)]) - 1)

                new_neighbour = (randint(0, self.L - 1), randint(0, self.L - 1))

                # dont connect to urself
                while new_neighbour == (i, j):
                    new_neighbour = (randint(0, self.L - 1), randint(0, self.L - 1))

                adj[(i, j)][index_to_change] = new_neighbour
    
    def get_max_cell(self, f: list[list[float]]):
        return max([max(row) for row in f])

    # gives a list[LxL], multiple calls allowed to simulate different alpha and f_th
    def simulate(self, STEPS: int):
        f = self._deepcopy_2d(self.f_0)
        adj = {(i, j): self._deepcopy_list_tuple(self.adj[(i, j)]) for i, j in self.adj}

        # Open boundary conditions
        for i in range(self.L):
            f[0][i] = 0
            f[self.L - 1][i] = 0
            f[i][0] = 0
            f[i][self.L - 1] = 0

        step_list: list[list[list[float]]] = [self._deepcopy_2d(f)] # add initial as a frame
        magnitudes: list[float] = [0]
        sizes: list[int] = [0]

        for step in range(STEPS):
            print(f"===== STEP {step} =====")
            active_queue = deque()

            max_cell = self.get_max_cell(f)
            
            delta = self.F_TH - max_cell

            for i in range(self.L):
                for j in range(self.L):
                    f[i][j] = min(self.F_TH, f[i][j] + delta)
                    if f[i][j] >= self.F_TH:
                        active_queue.append((i,j))

            earthquake_size = 0
            while len(active_queue) > 0:
                (ui, uj) = active_queue.popleft()
                fi = f[ui][uj]

                # f[ui][uj] = 0

                for (vi, vj) in adj[(ui,uj)]:
                    if f[vi][vj] < self.F_TH:
                        f[vi][vj] = min(f[vi][vj]+self.ALPHA*fi, self.F_TH)
                        if f[vi][vj] >= self.F_TH:
                            active_queue.append((vi,vj))
                earthquake_size += 1

            m = 0

            if earthquake_size > 0:
                print(f"EARTHQUAKE OF SIZE {earthquake_size} OCCURRED")
                m = log(earthquake_size)
                print(f"Earthquake Magnitude {m}")
            
            for i in range(self.L):
                for j in range(self.L):
                    if f[i][j] >= self.F_TH:
                        f[i][j] = 0

            step_list.append(self._deepcopy_2d(f))
            sizes.append(earthquake_size)
            magnitudes.append(m)

            # rewiring at random
            self.rewire(adj)
        
        
        return step_list, magnitudes, sizes

In [ ]:
# paramaters based on Greco et al.

L = 40
ALPHA = 0.21
F_TH = 1
STEPS = 2000
# set to random values initially
f: list[list[float]] = [[random() for _ in range(L)] for _ in range(L)]

ofc_random = OFCModel(L, ALPHA, F_TH, f, 2)
steps, magnitudes, sizes = ofc_random.simulate(STEPS)

print(max(magnitudes))

### Plot the generated model states

In [ ]:
import plotly.graph_objects as go

def plot_frames(steps: list[list[list[float]]], magnitudes: list[float]):
    frames = [
        go.Frame(data=go.Heatmap(z=s, zmin=0, zmax=1), name=f"Step {i}, Magnitude {magnitudes[i]}")
        for i, s in enumerate(steps)
    ]

    return go.Figure(data=frames[0].data, frames=frames).update_layout(
        updatemenus=[
            {
                "buttons": [{"args": [None, {"frame": {"duration": 100, "redraw": True}}],
                            "label": "Play", "method": "animate",},
                            {"args": [[None],{"frame": {"duration": 0, "redraw": False},
                                            "mode": "immediate", "transition": {"duration": 0},},],
                            "label": "Pause", "method": "animate",},],
                "type": "buttons",
            }
        ],
        # iterate over frames to generate steps... NB frame name...
        sliders=[{"steps": [{"args": [[f.name],{"frame": {"duration": 0, "redraw": True},
                                                "mode": "immediate",},],
                            "label": f.name, "method": "animate",}
                            for f in frames],}],
        height=800,
        width=800,
        title_x=0.5,
    )

In [ ]:
plot_frames(steps, magnitudes)

In [ ]:
import plotly.express as px

In [ ]:
df_size =  pd.DataFrame.from_dict({
    'seismic_event': [-1] + [i for i in range(STEPS)],
    'size': sizes,
})

fig = px.bar(df_size, x='seismic_event', y='size')
fig.update_traces(marker_line_width=0)
fig.show()

## Vulnerability Assessment

The synthetic data generated by the modified OFC model shall give a size $S$ which is the energy released by the entire seismic event, this is determined through the total amount of 'activated' $F_{th} = 1$ sites during the simulation.

Adapting the methods of Greco et al., (2019)
Since S is the energy of the event, the Magnitude $M$ can be determined through
$ M = \ln{S} $ From (Greco et al., 2019), this can be converted to Macroseismic intensity through, $I(M) = 1.71 M - 1.02$

With this, with we can use the intensity and refer to the Modified Mercalli Scale to determine the amount of damage that will be experienced towards that area over the span of 10 days.

In [ ]:
df_magnitudes =  pd.DataFrame.from_dict({
    'seismic_event': [-1] + [i for i in range(STEPS)],
    'magnitude': magnitudes,
})

fig = px.bar(df_magnitudes, x='seismic_event', y='magnitude')
fig.update_traces(marker_line_width=0)
fig.show()

In [ ]:
df_intensities =  pd.DataFrame.from_dict({
    'seismic_event': [-1] + [i for i in range(STEPS)],
    'intensity': [max(0, int(1.71 * magnitude - 1.02)) for magnitude in magnitudes], # 0 for negative intensities
})

fig = px.bar(df_intensities, x='seismic_event', y='intensity')
fig.update_traces(marker_line_width=0)
fig.show()

## Accuracy Testing through Spatio-temporal Analysis

To determine the accuracy of the synthetic data, temporal and spatial analysis will be performed through by generating a probability distribution for both groups of data.

To generate the probability distribution, the following formulae will be followed which were based on the integral form of Tsallis' entropy (Ferreira et al., 2022).

### For distance

$ P_≥(r) = \int_r^\infty P_q (r') d r' = e_q (-\beta_s r) $


Where $\beta_s$ is a scale constant. And $r$ is the distance of some earthquake from the epicenter.

### For time

$ P_≥(t) = \int_r^\infty P_q (t') d t' = e_q (-\beta_s t) $

With $\beta_t$ as a scale constant. And $t$ is the time interval between two earthquakes (calm time) (Ferreira et al., 2022).

# Results

So far we've been able to generate the Synthetic Earthquake data based on the parameters of Greco et al. 2019, that simulates multiple seismic events over a period, notably including multiple magnitude 4-5 earthquakes, similar to our chosen period in Davao within our dataset.

This gives us a sense of what magnitudes, intensity and 'energy' has been released during a certain period. This shall be correlated later on with the real data to determine 'when' exactly is this period in relation to the synthetic time steps generated.

In [ ]:
fig = px.bar(df_size, x='seismic_event', y='size')
fig.update_traces(marker_line_width=0)
fig.show()

fig = px.bar(df_magnitudes, x='seismic_event', y='magnitude')
fig.update_traces(marker_line_width=0)
fig.show()

fig = px.bar(df_intensities, x='seismic_event', y='intensity')
fig.update_traces(marker_line_width=0)
fig.show()

# Questions

1. For Synthetic Data Generation,
- The study we found currently refines a parameter $f_b$ by only continually adding 0.1 until the magnitude of the $L \times L$ matrix matches the magnitude formed by the original event with some error, 
- Are we allowed to perform a different method similar to newton iterations by adding delta, starting with 0.1, while the error keeps decreasing, when it increases after adding delta, halve it then go 'back' until the error decreases, repeat until we reach an error of epsilon

This $f_b$ is used to help the L x L matrix made to match the magnitude of the real world condition, to better mimic the earthquake in Davao, this method is applied by (Greco et al., 2019)



# References
Boeing, G. (2014). Clustering to Reduce Spatial Data Set Size. https://geoffboeing.com/2014/08/
clustering-to-reduce-spatial-data-set-size/

Ferreira, D., Ribeiro, J., Oliveira, P., Pimenta, A., Freitas, R., Dutra, R., Papa, A., & Mendes, J. (2022). 
Spatiotemporal analysis of earthquake occurrence in synthetic and worldwide data. Chaos 
Solitons & Fractals, 165, 112814. https://doi.org/10.1016/j.chaos.2022.112814

Greco, A., Pluchino, A., Barbarossa, L., Barreca, G., Caliò, I., Martinico, F., & Rapisarda, A. (2019). A 
New Agent-Based Methodology for the Seismic Vulnerability Assessment of Urban Areas. ISPRS 
International Journal of Geo-Information, 8(6). https://doi.org/10.3390/ijgi8060274